## Optional - make restraints
This will make restraints and generate configs for them.
Restraints between molecules has to be done using pull code added to MDP file.
See here http://www.mdtutorials.com/gmx/umbrella/05_pull.html
Index file need to be generated
```
---Example of pull code below ---
; Pull code
pull                    = yes
pull_ncoords            = 1         ; only one reaction coordinate
pull_ngroups            = 2         ; two groups defining one reaction coordinate
pull_group1_name        = Chain_A
pull_group2_name        = Chain_B
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = N N Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = yes       ; define initial COM distance > 0
pull_coord1_rate        = 0.01      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
```

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from myimports import *
display(Markdown(descr))

_ColormakerRegistry()


# MD simulations of H2A-H2B with 30 bp of DNA, tails truncated
- AMBER14SB force field
- 150 mM NaCl
- box 2nm


Folder set to:  h2a-h2b_tm_30DNA
Project name:  h2a-h2b_tm_30DNA
SSH host set to: newton


In [3]:
import MDAnalysis.analysis.hbonds
import MDAnalysis.analysis.distances


nucl=mda.Universe("GMX_system/sys_ref.pdb")


sel_end1_1="(segid I and resnum -60 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"
sel_end1_2="(segid J and resnum 60 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"

sel_end2_1="(segid I and resnum -31 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"
sel_end2_2="(segid J and resnum 31 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"

N=nucl.select_atoms("(segid I and resnum -31) or (segid J and resnum 31)")


E1NI=nucl.select_atoms(sel_end1_1)
E1NJ=nucl.select_atoms(sel_end1_2)

d1=MDAnalysis.analysis.distances.dist(E1NI,E1NJ)[2][0]/10.
print(d1)

E2NI=nucl.select_atoms(sel_end2_1)
E2NJ=nucl.select_atoms(sel_end2_2)

E2NI.atoms
d2=MDAnalysis.analysis.distances.dist(E2NI,E2NJ)[2][0]/10.
print(d2)

w=nv.show_mdanalysis(N,gui=False)
#w.add_representation( "label", ".N9", color="grey" )
w

0.8770063210397921
0.9302067589010917


NGLWidget()

In [4]:
e1ni=E1NI.atoms[0].id
e1nj=E1NJ.atoms[0].id
e2ni=E2NI.atoms[0].id
e2nj=E2NJ.atoms[0].id



Let's generate distance restraints
http://manual.gromacs.org/current/manual-2018.4.pdf


In [5]:
%%bash -s "$d1" "$d2" "$e1ni" "$e1nj" "$e2ni" "$e2nj" 
cat > GMX_system/pull.mdp <<!
; Pull code
pull                    = yes
pull_ncoords            = 2         ; only one reaction coordinate
pull_ngroups            = 4         ; two groups defining one reaction coordinate
pull_group1_name        = a_$3
pull_group2_name        = a_$4
pull_group3_name        = a_$5
pull_group4_name        = a_$6
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = Y Y Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = no       ; define initial COM distance > 0
pull_coord1_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord1-init        = $1

pull_coord2_type        = umbrella  ; harmonic potential
pull_coord2_geometry    = distance  ; simple distance increase
pull_coord2_dim         = Y Y Y     ; pull along z
pull_coord2_groups      = 3 4       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord2_start       = no       ; define initial COM distance > 0
pull_coord2_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord2_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord2-init        = $2
!

In [6]:
!cat GMX_system/pull.mdp


; Pull code
pull                    = yes
pull_ncoords            = 2         ; only one reaction coordinate
pull_ngroups            = 4         ; two groups defining one reaction coordinate
pull_group1_name        = a_11
pull_group2_name        = a_1888
pull_group3_name        = a_937
pull_group4_name        = a_968
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = Y Y Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = no       ; define initial COM distance > 0
pull_coord1_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord1-init        = 0.8770063210397921

pull_coord2_type        = umbrella  ; harmonic potential
pull_coord2_geometry    = distance  ; simple distance increase
pull_coord2_dim         =

In [7]:
%%bash -s "$d1" "$d2" "$e1ni" "$e1nj" "$e2ni" "$e2nj" "$set_gmx_cmd"
$7
cd GMX_system

gmx make_ndx -f init_solv_ions.pdb -n index.ndx -o index.ndx <<!
a $3
a $4
a $5
a $6
q
!

Going to read 1 old index file(s)

  0 System              : 86212 atoms
  1 DNA                 :  1908 atoms
  2 NA                  :   116 atoms
  3 CL                  :    73 atoms
  4 Protein             :  2998 atoms
  5 Protein-H           :  1463 atoms
  6 C-alpha             :   186 atoms
  7 Backbone            :   558 atoms
  8 MainChain           :   742 atoms
  9 MainChain+Cb        :   917 atoms
 10 MainChain+H         :   925 atoms
 11 SideChain           :  2073 atoms
 12 SideChain-H         :   721 atoms
 13 Prot-Masses         :  2998 atoms
 14 non-Protein         : 83214 atoms
 15 Water               : 81117 atoms
 16 SOL                 : 81117 atoms
 17 non-Water           :  5095 atoms
 18 Ion                 :   189 atoms
 19 NA                  :   116 atoms
 20 CL                  :    73 atoms
 21 Water_and_ions      : 81306 atoms
 22 !Water_and_ions     :  4906 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom 

                     :-) GROMACS - gmx make_ndx, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
    

In [8]:
%%bash
##Patch mdp files
cp GMX_system/pull.mdp MDProtocols/
cat MDProtocols/pull.mdp >> MDProtocols/1_minim.mdp
cat MDProtocols/pull.mdp >> MDProtocols/2_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/3_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/4_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/5_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/6_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/7_prod.mdp
cat MDProtocols/pull.mdp >> MDProtocols/bench.mdp

In [9]:
!cat MDProtocols/pull.mdp >> MDProtocols/7_prod_10ps_out.mdp

